In [1]:
!pwd

/home/dell/af/Pred_props


In [3]:
from unimol_tools import MolTrain, MolPredict
import pandas as pd
import numpy as np
import math
from sklearn.metrics import mean_absolute_error, r2_score
from scipy.stats import spearmanr
import os

In [4]:
# 更改工作目录到 `af` 目录
os.chdir('/home/dell/af')

# 验证当前工作目录
print(os.getcwd())

/home/dell/af


In [5]:
weights_path = r"./Unimol_weights/LUMO/0824_dataset_lr_1e-4"
data_path = r"./Data4pred/sam4tl_0904.csv"
save_path = r"./Data4pred/sam4tl_0904_LUMO.csv"
input_csv = r"./tmp.csv"

In [6]:
data = pd.read_csv(data_path)
data.head(5)

,SMILES,scaffold_smiles
0,O=P(O)(O)CCCCn1c2cc(Br)c3ccccc3c2c2c3ccccc3c(B...,c1ccc2c(c1)ccc1[nH]c3ccc4ccccc4c3c12
1,O=P(O)(O)CCCCn1c2ccccc2c2c3ccccc3ccc21,c1ccc2c(c1)ccc1[nH]c3ccccc3c12
2,O=P(O)(O)CCCCn1c2ccccc2c2ccccc21,c1ccc2c(c1)[nH]c1ccccc12
3,O=P(O)(O)CCCCn1c2ccc3ccccc3c2c2c3ccccc3ccc21,c1ccc2c(c1)ccc1[nH]c3ccc4ccccc4c3c12
4,COC1=CC=C(C=C1)N(C1=CC=C(OC)C=C1)C1=CC=C(C=C1)...,c1ccc(N(c2ccccc2)c2ccc(-c3ccc4c(c3)Sc3cc(-c5cc...


In [7]:
pd.DataFrame(data.iloc[:,0]).to_csv(input_csv, index=False)

In [8]:
clf = MolPredict(load_model=weights_path)

In [9]:
# 基于SMILES的文件输入模式的预测 
test_pred = clf.predict(data=input_csv)

2024-09-13 16:39:28 | unimol_tools/data/conformer.py | 89 | INFO | Uni-Mol Tools | Start generating conformers...
295it [00:18, 16.08it/s]
2024-09-13 16:39:46 | unimol_tools/data/conformer.py | 93 | INFO | Uni-Mol Tools | Failed to generate conformers for 0.00% of molecules.
2024-09-13 16:39:46 | unimol_tools/data/conformer.py | 95 | INFO | Uni-Mol Tools | Failed to generate 3d conformers for 2.03% of molecules.
2024-09-13 16:39:47 | unimol_tools/models/unimol.py | 120 | INFO | Uni-Mol Tools | Loading pretrained weights from /home/dell/miniconda3/envs/unimol/lib/python3.10/site-packages/unimol_tools-0.1.0.post2-py3.10.egg/unimol_tools/weights/mol_pre_no_h_220816.pt
2024-09-13 16:39:48 | unimol_tools/models/nnmodel.py | 206 | INFO | Uni-Mol Tools | start predict NNModel:unimolv1
2024-09-13 16:39:49 | unimol_tools/tasks/trainer.py | 300 | INFO | Uni-Mol Tools | load model success!
2024-09-13 16:39:52 | unimol_tools/tasks/trainer.py | 300 | INFO | Uni-Mol Tools | load model success!
2024-

In [10]:
os.remove(input_csv)

In [11]:
test_results = pd.DataFrame({
    "smiles": clf.datahub.data["smiles"],
    "pred": test_pred.flatten(),
    })
test_results.head(10)

,smiles,pred
0,O=P(O)(O)CCCCn1c2cc(Br)c3ccccc3c2c2c3ccccc3c(B...,-1.411726
1,O=P(O)(O)CCCCn1c2ccccc2c2c3ccccc3ccc21,-0.930162
2,O=P(O)(O)CCCCn1c2ccccc2c2ccccc21,-0.563643
3,O=P(O)(O)CCCCn1c2ccc3ccccc3c2c2c3ccccc3ccc21,-1.095771
4,COC1=CC=C(C=C1)N(C1=CC=C(OC)C=C1)C1=CC=C(C=C1)...,-0.651735
5,COC1=CC=C(C=C1)N(C1=CC=C(OC)C=C1)C1=CC=C(C=C1)...,-0.561071
6,COC1=CC=C(C=C1)N(C1=CC=C(OC)C=C1)C1=CC=C(C=C1)...,-0.622011
7,O=P(O)(O)CCN1c2ccc(Br)cc2Oc2cc(Br)ccc21,-0.791403
8,O=P(O)(O)CCN1c2ccc(Br)cc2Sc2cc(Br)ccc21,-0.988199
9,O=P(O)(O)CCN1c2ccc(Br)cc2[Se]c2cc(Br)ccc21,-0.916014


In [12]:
test_results.to_csv(save_path, index=False)